# Calibration Example

Example for using PySimpleFOC python API.

This example can be used to compute and plot motor calibration values.
To use it, your SimpleFOC code has to include support for packet based IO and telemetry.

The code controls the motor in open loop mode. Ideally, the arduino code should run the initFOC() method to determine the correct motor orientation, and this notebook will read it from the driver. Alternatively you can set the direction in advance using parameter below.

A simple sketch compatible with this notebook can be found here TODO link to sample!


## Imports and Constants

Change the constants below to control the experiments parameters.

In [ ]:
%matplotlib ipympl
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import serial, time, math
import numpy as np
import simplefoc, simplefoc.packets
from simplefoc.registers import SimpleFOCRegisters as regs
from rx import operators as ops

PORT="/dev/cu.usbmodem1441103"
BAUD=921600*2
PROTOCOL=simplefoc.packets.ProtocolType.ascii

ROTATIONS=5
VOLTAGE=4.0
POLE_PAIRS=11
VELOCITY=3.14
DOWNSAMPLE=4
DIRECTION=simplefoc.Direction.UNKNOWN

_2PI = 2.0 * math.pi
_3PI_2 = 3.0 * math.pi / 2.0

## Setup
Run this once.

In [ ]:
sf = simplefoc.packets.serial(PORT, BAUD, PROTOCOL)
sf.connect()
tele = sf.telemetry()
tele.set_registers([regs.REG_ANGLE, regs.REG_POSITION],[1,1])
motor = sf.motor(1)
motor.set_mode(simplefoc.MotionControlType.velocity_openloop, simplefoc.TorqueControlType.voltage)
motor.set_register(regs.REG_ZERO_OFFSET, 0)
motor.set_register(regs.REG_ZERO_ELECTRIC_ANGLE, 0)
motor.set_limits(max_voltage=VOLTAGE)
if DIRECTION == simplefoc.Direction.UNKNOWN:
    direction = motor.get_register(regs.REG_SENSOR_DIRECTION)
    if direction>1:
        direction = -1
    DIRECTION = simplefoc.Direction(int(direction))
    print("Direction read from motor: ", DIRECTION.name)

limit = (ROTATIONS + 1.5) * _2PI
crossing_val = _3PI_2 if DIRECTION==simplefoc.Direction.CW else (-_3PI_2+_2PI)

def compute_el(x):
    el = (x.values[0] * POLE_PAIRS * DIRECTION.value) % _2PI
    if el<0:
        el += _2PI
    x.values.append(el)
    if x.values[1] >= (2**31):
        x.values[1] -= (2**32)
    return x.values

def transform_telemetry(w):
    if DIRECTION==simplefoc.Direction.CW:
        crossing = ((w[0][3] <= crossing_val and w[1][3] >= crossing_val) or (w[1][3] <= crossing_val and w[2][3] >= crossing_val))
    else:
        crossing = ((w[0][3] >= crossing_val and w[1][3] <= crossing_val) or (w[1][3] >= crossing_val and w[2][3] <= crossing_val))
    result = [
        w[1][1], # measured rotation
        w[1][0], # open loop angle
        w[1][2], # measured mechanical angle
        w[1][3], # electrical angle
        1 if crossing else 0, # 1 if this is (next to) a zero-crossing
    ]
    return result

## Data collection

Run to collect data.

In [ ]:
print("Collecting data at velocity", VELOCITY, "for", ROTATIONS, "rotations.")
print("This will take about", limit/VELOCITY, "seconds")

motor.enable()
motor.set_target(VELOCITY)
tele.set_downsample(DOWNSAMPLE)

data = []

try:
    data = tele.observable().pipe(
        ops.map(lambda x: compute_el(x)),
        ops.take_while(lambda x: abs(x[1]*_2PI + x[2])<limit),
        ops.buffer_with_count(3,1),
        ops.filter(lambda x: len(x)==3),
        ops.map(lambda x: transform_telemetry(x)),
        ops.filter(lambda x: (abs(x[0])>=1 and abs(x[0])<=ROTATIONS)),
        ops.to_list(),
        ops.timeout(limit/VELOCITY*1.5)
    ).run()
finally:
    motor.set_target(0.0)
    tele.set_downsample(10000)
    motor.disable()


if len(data) < 1000:
    print("Not enough data collected.")
else:
    print("Collected", len(data), "samples")
    print(len(data)/ROTATIONS, "samples per rotation")
    print((ROTATIONS*360)/len(data), "degrees rotation per sample")

## Plot zero crossings

The electrical zero crossings, shown plotted against the sensor's full rotation.

In [ ]:
linetheta = []
for i in range(0, len(data)):
    if data[i][4] == 1:
        linetheta.append(data[i][2])
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.grid(True, axis='x')
ax.grid(False, axis='y')
ax.set_rmax(6)
ax.set_rmin(0)
ax.set_theta_zero_location('N')
ax.set_title("Zero crossings", va='bottom')
ax.vlines(linetheta, 4, 5)
plt.show()

## Plot zero crossings
**Mapped to the first electrical revolution**

The electrical zero crossings, all mapped to the first electrical revolution, and the average. Use this average as your electrical zero calibration value.

In [ ]:
linetheta2 = np.array(linetheta) % (_2PI / POLE_PAIRS)
avg = np.mean(linetheta2)
min = np.min(linetheta2)
max = np.max(linetheta2)
avg_deg = -avg/_2PI*360
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.set_rmax(6)
ax.set_rmin(0)
ax.grid(True, axis='x')
ax.grid(False, axis='y')
diff = abs( np.max([avg - min, max - avg]) ) * 1.25 * 360 / _2PI
if diff < 22.5:
    diff = 22.5
ax.set_thetamin(-avg_deg-diff)
ax.set_thetamax(-avg_deg+diff)
ax.set_theta_zero_location('N', avg_deg)
ax.set_title("Zero crossings", va='bottom')
ax.vlines(linetheta2, 4, 5)
plt.show()
print("motor.zero_electric_angle={}f;".format(avg))
print("motor.sensor_direction=Direction::"+DIRECTION.name+";")

## Plot linearity

Plots the measured sensor position (y axis) against the commanded open-loop angle.

In [ ]:
xdata = []
ydata = []
curseries = -1
for i in range(0, len(data)):
    if i==0 or data[i][0] != data[i-1][0]:
        curseries += 1
        adj = 0
        xdata.append([])
        ydata.append([])
    x = data[i][1]
    if i>0 and abs(x - data[i-1][1])>math.pi:
        adj += _2PI if data[i-1][1]>x else -_2PI
    xdata[curseries].append(x+adj)
    ydata[curseries].append(data[i][2])
fig, ax = plt.subplots()
for i in range(0, curseries+1):
    ax.plot(xdata[i], ydata[i])
#plt.set_title("Linearity", va='bottom')
plt.show()